# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [23]:
import pandas as pd
import numpy as np

#1

airbnb = pd.read_csv("./data/airbnb_hw.csv")

airbnb["Price_clean"] = (
    airbnb["Price"]
    .str.replace(",", "", regex=False)
    .astype(float)
)

print(airbnb["Price_clean"].head(10))

# removed commas from numbers over 999, changed values to float to allow for numeric calculations

missing_prices = airbnb["Price_clean"].isna().sum()
print("Missing vals:", missing_prices)

# 0 missing values in cleaned column

0    145.0
1     37.0
2     28.0
3    199.0
4    549.0
5    149.0
6    250.0
7     90.0
8    270.0
9    290.0
Name: Price_clean, dtype: float64
Missing vals: 0


In [43]:
#2

police = pd.read_csv("./data/mn_police_use_of_force.csv")
police["subject_injury"].value_counts(dropna=False)

police["subject_injury_clean"] = (
    police["subject_injury"]
    .str.strip()
    .str.lower()
    .replace({"none": "no"})
)

police["subject_injury_clean"] = police["subject_injury_clean"].map({
    "yes": "Yes",
    "no": "No"
})

police["subject_injury_clean"].value_counts(dropna=False)

prop_missing = police["subject_injury_clean"].isna().mean()
print(prop_missing)

# the proportion of missing values is roughly 76.19 percent
# this number is concerning it is difficult to draw reliable conclusions from a data set with such a high proportion of missing values


0.7619342359767892


In [47]:
ct = pd.crosstab(police["subject_injury_clean"], police["force_type"], margins=True)
print(ct)

# patterns that stick out are higher number of data points in the categories Bodily Force and Taser and lower number in Baton and Firearm
# more missing data/limits in data collection in more extreme forms such as Firearm

force_type            Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury_clean                                                    
No                        0          1093                131        2   
Yes                       2          1286                 41        0   
All                       2          2379                172        2   

force_type            Gun Point Display  Improvised Weapon  \
subject_injury_clean                                         
No                                   33                 34   
Yes                                  44                 40   
All                                  77                 74   

force_type            Less Lethal Projectile  Police K9 Bite  Taser   All  
subject_injury_clean                                                       
No                                         1               2    150  1446  
Yes                                        2              44    172  1631  
All                

In [56]:
#3

justice = pd.read_parquet("./data/justice_data.parquet")

justice["WhetherDefendantWasReleasedPretrial"].value_counts(dropna=False)

justice["ReleasedPretrial_clean"] = justice["WhetherDefendantWasReleasedPretrial"].replace(9, np.nan)

justice["ReleasedPretrial_clean"].value_counts(dropna=False)

justice["ReleasedPretrial_str"] = justice["ReleasedPretrial_clean"].map({
    1.0: "Yes",
    0.0: "No",
    np.nan: np.nan
})

justice["ReleasedPretrial_str"].value_counts(dropna=False)

,count
ReleasedPretrial_str,
Yes,19154
No,3801
NaN,31


In [59]:
#4

justice[["ImposedSentenceAllChargeInContactEvent",
         "SentenceTypeAllChargesAtConvictionInContactEvent"]].head(20)

justice["ImposedSentence_clean"] = justice["ImposedSentenceAllChargeInContactEvent"]

justice.loc[
    justice["SentenceTypeAllChargesAtConvictionInContactEvent"].isna(),
    "ImposedSentence_clean"
] = np.nan

missing_sentences = justice["ImposedSentence_clean"].isna().sum()

# replaced missing sentences with NaN when corresponding sentence type was missing, marked missing vals for cases without conviction

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)